In [2]:
import pandas as pd
import numpy as np

In [3]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [4]:
movies = movies.merge(credits, on= 'title')

In [5]:
movies.head(1)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [6]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [7]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [8]:
movies.dropna(inplace = True)
movies.duplicated().sum()

0

In [9]:
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
        
    return L

In [10]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [11]:
def convert3(obj):
    L = []
    ctr = 0
    for i in ast.literal_eval(obj):
        if ctr != 3:
            L.append(i['name'])
            ctr= ctr + 1
        else:
            break

    return L

In [12]:
movies['cast'] = movies['cast'].apply(convert3)

In [13]:
def fetch(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break

    return L

In [14]:
movies['crew'] = movies['crew'].apply(fetch)

In [15]:
movies['overview'] = movies['overview'].apply(lambda x : x.split())

In [16]:
movies['genres'] = movies['genres'].apply(lambda x : [i.replace((" "), "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x : [i.replace((" "), "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x : [i.replace((" "), "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x : [i.replace((" "), "") for i in x])

In [17]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [18]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies = movies[['movie_id','title', 'tags']]

In [19]:
movies['tags'] = movies['tags'].apply(lambda x : " ".join(x))
movies['tags'] = movies['tags'].apply(lambda x : x.lower())

In [20]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [21]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y) 
        

In [22]:
movies['tags'] = movies['tags'].apply(stem)

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 50000, stop_words = 'english')

In [24]:
vectors = cv.fit_transform(movies['tags']).toarray()

In [25]:
from sklearn.metrics.pairwise import cosine_similarity


In [26]:
similarity_matrix = cosine_similarity(vectors)

In [27]:
def recommend(movie):
    movie_index = movies[movies['title'] == movie].index[0]
    distances = similarity_matrix[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x : x[1])[1:6]

    for i in movies_list:
        print(movies.iloc[i[0]].title)

In [28]:
recommend('Batman')

Batman
Batman & Robin
The R.M.
Batman Begins
Batman Returns


In [30]:
import pickle
pickle.dump(movies.to_dict(), open('movies_dict.pkl', 'wb'))
pickle.dump(similarity_matrix, open('similarity.pkl', 'wb'))
import json